# Práctica 2.A Toma de contacto con AIMA

La práctica está organizada en 3 partes. En la primera se muestra a través de ejemplos cómo se implementan algunos problemas clásicos como el de las jarras o el problema del ocho puzzle. En la segunda parte se muestra el uso de los algoritmos de búsqueda. En la tercera parte aprenderemos a medir las propiedades de los algoritmos.
En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios. 

## Parte I: Representación de problemas de espacios de estados.

### El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. En esta parte en vez de importarla la hemos copiado aquí para la explicación.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros.

In [96]:
from search import *

In [32]:
   class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [33]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [34]:
p =Jarras()
p.initial

(0, 0)

In [35]:
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [36]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [37]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [38]:
p.goal_test(p.initial)

False

# Representa el problema del puzzle de 8

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. Tenlo en cuenta al hacer las pruebas. 

### EJERCICIO 1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 

In [39]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        
        return accs
        ### EJERCICIO 1.1. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        
        
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        elif accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        elif accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        elif accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        
        
       ### EJERCICIO 1.2. COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1

### Ejercicio 1.3. Probar los siguientes ejemplos que se pueden ejecutar una vez se ha definido la clase:

In [40]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [41]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [42]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [43]:
p8.result(p8.initial,"Mover hueco abajo") # Da error porque no se puede bajar en el estado inicial

IndexError: list index out of range

In [44]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [45]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

1

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

### Usaremos búsqueda en anchura y en profundidad para encontrar soluciones tanto al problema de las jarras como al problema del ocho puzzle con distintos estados iniciales.

In [46]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

In [47]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [48]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### Ejercicio 2. Prueba los algoritmos de búsqueda ciega con el puzzle de 8

In [49]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [50]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [ ]:
#breadth_first_tree_search(EightPuzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
# Busqueda en anchura sin control de repetidos.  
# Busqueda en anchura es completo.. debería terminar...  ¿qué crees que está pasando?

In [ ]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

In [51]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [52]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['UP', 'LEFT', 'UP', 'LEFT', 'DOWN', 'RIGHT', 'RIGHT', 'DOWN']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [53]:
depth_first_graph_search(estado).solution()

KeyboardInterrupt: 

In [54]:
breadth_first_graph_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

#### En este ejercicio se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones:

---------------------------------
--------------------------------------------------------------------------------------------------
 --------------------------
 -----------------------------------------------------
 -------------------------
 
**********************************************************************************



### Ejercicio 3: definir al menos las siguientes funciones heurísticas para el 8 puzzle:
* linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
* manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
* max_heuristic(node: maximo de las dos anteriores
* sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan

In [55]:
# Heuristicas para el 8 Puzzle 

def linear(node):
    #goal = node.state.goal
    mal_colocadas = 0
    goal = node.state.goal
    
    for x in range(0,8):
        if (node.state[x] != goal[x]) : mal_colocadas += 1
            
    return mal_colocadas

def distancia_manhattan(node1, node2):
    return (abs(node1[0] - node2[0]) + abs(node1[1] - node2[1]))

def corresponde (num):
    tabla = ((0,0),(0,1), (0,2), (1,0), (1,1), (1,2),(2,0),(2,1),(2,2))
    node = (2,1)
    
    if(num == 0) : node = tabla[8]
    elif (num == 1) : node = tabla[0]
    elif (num == 2) : node = tabla[1]
    elif (num == 3) : node = tabla[2]
    elif (num == 4) : node = tabla[3]
    elif (num == 5) : node = tabla[4]
    elif (num == 6) : node = tabla[5]
    elif (num == 7) : node = tabla[6]
    
    return node

def manhattan(node):
    
    state = node.state
    lista = 0
    mht = 0
    inicial = (0,0)
    objetivo = (0,0)
    
    for x in range(0, 2):
        for y in range (0,2):
            inicial = (x,y)
            objetivo = corresponde(state[lista])
            mht += distancia_manhattan(inicial, objetivo)
        
        lista+=1
    
    return mht

def sqrt_manhattan(node):
    
    mhd = manhattan(node)
    
    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)
        

### Ejercicio 4. 
Usar las implementaciones de los algoritmos que correspondan a búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema del 8 puzzle para el siguiente estado inicial y comparar los costes temporales usando %timeit y comentar los resultados.

              +---+---+---+
              | 2 | 4 | 3 |
              +---+---+---+
              | 1 | 5 | 6 |
              +---+---+---+
              | 7 | 8 | H |
              +---+---+---+


In [56]:
puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
astar_search(puzle).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [58]:
astar_search(puzle,manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [59]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [60]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [61]:
astar_search(puzle,linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [62]:
astar_search(puzle,max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

¿Has notado diferencias en los tiempos de ejecución? Vamos a medirlo.Aunque las heurísticas no afectan a la solución obtenida sí hay diferencias importantes en el tiempo de cálculo
<br>

In [63]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [64]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

3.18 ms ± 4.39 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [65]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

158 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [66]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

392 ms ± 10.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [67]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

49 ms ± 126 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Escribe aquí tus conclusiones sobre qué heurística es mejor y por qué.

# ----------------------------------------------------------------------------
# ----------------------------------------------------------------------------


## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil.. como hemos comentado al principio solo algunos tableros tienen solucion.
En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando el número de inversiones. Si es impar el tablero no tiene solución. 

The solvability of a configuration can be checked by calculating the Inversion Permutation. If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.

In [68]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas, para resolver el 8-puzzle. 
# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """

        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        
        return inversion % 2 == 0        

In [78]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

In [21]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [22]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [82]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1, max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [77]:
astar_search(p8, linear).solution()

KeyboardInterrupt: 

In [73]:
astar_search(p8p, max_heuristic).solution()

['Mover hueco derecha']

In [24]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """

    p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
    if p8p.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [107]:
resuelve_ocho_puzzle(estado_inicial,astar_search,linear)
print("\n")
puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
resuelve_ocho_puzzle(puzzle_1,astar_search,linear)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 1. Nodos analizados: 2


Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: linear
Longitud de la solución: 8. Nodos analizados: 13


#### Ejercicio 5

#Intentar resolver usando las distintas búsquedas y en su caso, las distintas
#heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

          E1              E2              E3              E4
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    
     | 2 | 8 | 3 |   | 4 | 8 | 1 |   | 2 | 1 | 6 |   | 5 | 2 | 3 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 1 | 6 | 4 |   | 3 | H | 2 |   | 4 | H | 8 |   | H | 4 | 8 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
     | 7 | H | 5 |   | 7 | 6 | 5 |   | 7 | 5 | 3 |   | 7 | 6 | 1 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la
 siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo, y
 justificarlos con las distintas propiedades teóricas estudiadas.

-----------------------------------------------------------------------------------------
                                       E1           E2           E3          E4
    Anchura                            L=           L=           L=          L=
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA= 
   
    Profundidad                        L=           L=           L=          L=  
                                       T=           T=           T=          T= 
                                       NA=          NA=          NA=         NA= 
                                       
    Coste Uniforme                     L=           L=           L=          L=
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA= 
                                       
    Primero el mejor (linear)          L=           L=           L=          L=
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=
                                                                            
    Primero el mejor (manhattan)       L=           L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=
                                                                             
    A* (linear)                        L=           L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=
                                                                             
     A* (manhattan)                    L=           L=           L=          L= 
                                       T=           T=           T=          T=  
                                       NA=          NA=          NA=         NA=

 -----------------------------------------------------------------------------------------




### Ejercicio 6: [OPCIONAL] Definir una heurística más informada y completa la tabla anterior para ver cómo afecta al número de nodos generados por los algoritmos

In [456]:
# Proponemmos como heurística h3 la se obtiene sumando a la heurística
# manhattan una componente que cuantifica la "secuencialidad" en las
# casillas de un tablero, al recorrerlo en el sentido de las aguas del reloj
# ¿Es h3 admisble? Comprobar cómo se comporta esta heurística cuando se usa en
# A*, con cada uno de los estados anteriores. Comentar los resultados.
# Puedes proponer otra heuristica siempre que sea más informada que todas las heurísticas ya utilizadas.

         

# ---------------------------------------------------------------------------

